In [2]:
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import csv

In [3]:
urls = ['https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=']*3

In [4]:
urls

['https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=',
 'https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=',
 'https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=']

In [5]:
#list of urls
page = 1
while (page<=2):
    url_pos = page
    urls[url_pos] = urls[url_pos] + '&page=' + str(page + 1)
    page = page + 1

In [6]:
urls

['https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=',
 'https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=&page=2',
 'https://www.rottentomatoes.com/tv/squid_game/s01/reviews?type=&sort=&page=3']

In [7]:
#obtaining data
reviews=[]
for url in urls:
    client = uReq(url)
    page_html = client.read()
    client.close()
    page_soup = soup(page_html, "html.parser")
    reviews = reviews + [review.text for review in page_soup.find_all("div", {"data-qa": "review-quote"})]

In [8]:
len(reviews)

57

In [9]:
reviews

['\r\n                    The Squid Game is a wonderfully crafted suspenseful show that keeps you on the edge of your seat. But as the episodes move forward, the over dramatic elements take center stage and the whole thing loses steam scene after scene.\r\n                ',
 '\r\n                    The originality of the idea, the visual effects and of course, the acting are outstanding and incomparable to anything we have ever seen on television in our lives!\r\n                ',
 '\r\n                    Awesome!\r\n                ',
 "\r\n                    Netflix's breakout series depicts a world of violent and macabre individualism and desperation.\r\n                ",
 "\r\n                    Squid Game is many things, but above all it's top-notch junk TV, to be binged with swivel-eyed abandon.\r\n                ",
 '\r\n                    I don\'t know if anything is "111 million views instantly" good but at least Squid Game delivered on the hype, it\'s if nothing else

In [10]:
#csv with reviews
with open('rotten_reviews.csv', 'w') as myfile:
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
    wr.writerow(reviews)

After running the code above once, run from this point

In [11]:
import csv
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 

#from wordcloud import WordCloud, ImageColorGenerator
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Wilson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Wilson\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Wilson\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
with open('rotten_reviews.csv', 'r') as f:
  reader = csv.reader(f)
  reviews = list(reader)[0]

In [13]:
len(reviews)

57

In [14]:
stop_words = set(stopwords.words('english')) 
word_tokens=[]
for review in reviews:   
    word_tokens = word_tokens + word_tokenize(review) 
  
filtered_word_reviews = [w for w in word_tokens if not w in stop_words]

In [15]:
print(word_tokens[:10]) 
print(filtered_word_reviews[:10])

['The', 'Squid', 'Game', 'is', 'a', 'wonderfully', 'crafted', 'suspenseful', 'show', 'that']
['The', 'Squid', 'Game', 'wonderfully', 'crafted', 'suspenseful', 'show', 'keeps', 'edge', 'seat']
